In [ ]:
import itertools
import sys
from pathlib import Path

import h5py as h5
import numpy as np
import pandas as pd
import xarray as xr
from tqdm import tqdm

sys.path.append('../lib')
from local_paths import preproc_dir, analysis_dir, database_dir, cache_dir
from storage import get_storage_functions, quantize
from im_patches import get_patches_from_grid
from cross_val_pred import standardize, cv_split_by_image, cv_ridge_predict_eval

# Parameters

In [ ]:
#============================================================================
# session
#============================================================================
sess_name = 'sess_name'


#============================================================================
# spatiotemporal resolution of RF
#============================================================================
# temporal
t_aln  = 'sacc_on'  # 'sacc_on' or 'fix_on'
# response windows
# - one (long) window, if > 0
t_win  =   0
# - OTHERWISE, sliding window
t_pre  = 375
t_post = 375  # inclusive, but window must fit fully in range
t_step =  25

# spatial
xy_min  = -7  # dva
xy_max  =  7
xy_step =  1  # only meaningful if at least patch step size in cached reprs;
              # note that patch size is set in cached reprs


#============================================================================
# fixation/saccade selection
#============================================================================
# fixation criteria
ifix_sel        =    1    # 0: zeroth-fix only; 1: non-zeroth-fix only; otherwise: both
select_saccades = True    # must be False if ifix_sel is None or 0


#============================================================================
# modelling
#============================================================================
# ridge regularization
ridge_alpha = 100000

# cross-validation
n_splits    =      5
group_kfold =   True


#============================================================================
# paths
#============================================================================
proc_dir = preproc_dir

sdf_dir = preproc_dir
sdf_suffix = '-mwa_1' if t_win > 0 else '-mwa_50'  # default to no smoothing if using a response window

feat_dir = cache_dir + 'feats/vit_large_patch16_384/blocks.13.attn.qkv'
feat_suffix = '_as_2x2_in_0.50_steps'

unit_sel_path = None
latency_path = database_dir + 'per_unit_latency-fix_on.csv.gz'  # only used # if t_win > 0

output_dir = analysis_dir + 'model_perf_map'

# Check prereqs and params

In [ ]:
proc_path = Path(proc_dir) / (sess_name + '-proc.h5')
print('Loading shared processing from', proc_path)
proc_path = proc_path.expanduser()
assert proc_path.is_file()

sdf_path = Path(sdf_dir) / (sess_name + f'-sdf{sdf_suffix}.h5')
print('Loading spike density function from', sdf_path)
sdf_path = sdf_path.expanduser()
assert sdf_path.is_file()

with h5.File(proc_path, 'r') as f:
    im_w, im_h = im_size = f['stimulus/size_dva'][()]
feats_path = Path(feat_dir) / f'{im_w:.1f}x{im_h:.1f}{feat_suffix}.h5'
print('Loading cached model features from', feats_path)
feats_path = feats_path.expanduser()
assert feats_path.is_file()

if t_win > 0:
    print('Using per-unit latency from', latency_path)
    latency_path = Path(latency_path).expanduser()
    assert latency_path.is_file()

if unit_sel_path is not None:
    print('Loading unit selection from', unit_sel_path)
    unit_sel_path = Path(unit_sel_path).expanduser()
    assert unit_sel_path.is_file()
    unit_names = pd.read_csv(unit_sel_path).set_index('Session').loc[[sess_name]]['Unit'].values
else:
    unit_names = None

output_dir = Path(output_dir)
assert output_dir.expanduser().is_dir()
output_path = output_dir / (sess_name + '.h5')
print('Saving results to', output_path)
output_path = output_path.expanduser()

In [ ]:
analysis_name = 'model_perf_map'

if output_path.is_file():
    with h5.File(output_path, 'r') as f:
        try:
            if f[f'progress_report/{analysis_name}/all_done'][()].item():
                raise RuntimeError(f'{sess_name} has already been processed')
        except KeyError:
            pass

In [ ]:
fix_df = pd.read_hdf(proc_path, 'fixation_dataframe', 'r')
with h5.File(proc_path, 'r') as f:
    stim_folder_ = f['stimulus/folder'][()].decode()

imids = fix_df.groupby(['Image subdir', 'Image filename']).first().index
md5s = [Path(fn).stem for _, fn in imids]
md5_catalog = pd.DataFrame(index=imids, data=md5s, columns=['MD5'])

# Prepare parameters; save config

In [ ]:
with h5.File(proc_path, 'r') as f:
    random_seed = f['config/default_random_seed'][()]
print('random_seed:', random_seed)

In [ ]:
with h5.File(proc_path, 'r') as f:
    if select_saccades:
        assert ifix_sel == 1, 'if selecting saccades, must only include non-zeroth fixations'
        fix1_sel, fix2_sel = f['saccade_selection/fixation_indices'][()]
    else:
        assert t_aln == 'fix_on'
        fix2_sel = f['fixation_selection/fixation_indices'][()]
        if ifix_sel in (0, 1):
            m = 0 == fix_df.index.get_level_values(fix_df.index.names.index('Fixation'))[fix2_sel]
            if ifix_sel == 1:
                m = ~m
            fix2_sel = fix2_sel[m]

In [ ]:
assert t_aln in ('sacc_on', 'fix_on')
if t_win > 0:
    ts = np.array([t_win/2])  # placeholder; actual window varies with latency per unit
else:
    ts = np.arange(-t_pre, t_post+.1, t_step)
print(f'Response windows ({len(ts)}): {ts}')

rf_locs = np.arange(xy_min, xy_max+xy_step/10, xy_step)
print(f'RF locations ({len(rf_locs)}): {rf_locs}')
tb = (rf_locs[0]-xy_step/2, rf_locs[-1]+xy_step/2)
rf_extent = tb + tb

In [ ]:
save_results, add_attr_to_dset, check_equals_saved, link_dsets, copy_group = \
    get_storage_functions(output_path)

In [ ]:
group = analysis_name + '/'
save_results(group+'ts', ts, attrs=dict(unit='ms'))
save_results(group+'rf_locs', rf_locs, attrs=dict(unit='dva'))
save_results(group+'sdf_suffix', sdf_suffix)

group = analysis_name + '/config/'
save_results(group+'random_seed', random_seed)

group = analysis_name + '/config/time_windows/'
save_results(group+'t_aln', t_aln)
save_results(group+'t_win', t_win)
save_results(group+'t_pre', t_pre)
save_results(group+'t_post', t_post)
save_results(group+'t_step', t_step)
add_attr_to_dset(group, attrs=dict(unit='ms'))

group = analysis_name + '/config/retinotopic_locations/'
save_results(group+'xy_min', xy_min)
save_results(group+'xy_max', xy_max)
save_results(group+'xy_step', xy_step)
add_attr_to_dset(group, attrs=dict(unit='dva'))

group = analysis_name + '/config/fixation_selection/'
save_results(group+'ifix_sel', ifix_sel)
save_results(group+'select_saccades', select_saccades)

group = analysis_name + '/config/modelling/'
save_results(group+'n_splits', n_splits)
save_results(group+'ridge_alpha', ridge_alpha)
save_results(group+'group_kfold', group_kfold)

# For saccades, construct two types of control points
- Control 1: the 3rd vertex of two equilateral triangles anchored by fix 1 & 2 (previous and current), i.e., the L4/5 Langange points. One of the points is chosen:
  - If both points are within image bounds, choose randomly
  - If one and only one point is within image bounds, choose that
  - If both points are outside image bounds, choose the one closest to the image.
- Control 2: the midpoint between the previous and current fixation, i.e., the L1 point.

In [ ]:
xys_fix2 = fix_df.iloc[fix2_sel][['Relative X', 'Relative Y']].values.astype(float)
if select_saccades:
    xys_fix1 = fix_df.iloc[fix1_sel][['Relative X', 'Relative Y']].values.astype(float)

In [ ]:
if not select_saccades:
    n_cond = 1

else:
    n_cond = 4
    rg = np.random.default_rng(random_seed)

    # control point type 2: midpoint between fixs 1, 2
    xys_control2 = (xys_fix1 + xys_fix2) / 2

    # control point type 1
    # - get 3rd vertex of the equilateral triangle
    #   by rotating the saccade vector +/-60 degs
    vs = xys_fix2 - xys_fix1
    rmats = [
        np.array([[np.cos(a), -np.sin(a)], [np.sin(a), np.cos(a)]])
        for a in (np.pi/3, -np.pi/3)
    ]
    xys_l45 = xys_fix1 + np.array([(m @ vs.T).T for m in rmats])  # shape (n_fix, 2)

    # - check whether/how far points are out of image bounds
    d2b = np.array([
        [
            np.max([xys[:,j]-im_size[j]/2, -im_size[j]/2-xys[:,j]], axis=0)
            for j in range(2)
        ]
        for xys in xys_l45
    ])  # shape (2, 2, n_fix)
    oob = np.any(d2b >= 0, axis=1)  # shape (2, n_fix)

    # - choose one of the two candidate control points according to 1 of 3 conditions
    m0 = np.all(oob, axis=0)
    m2 = np.all(~oob, axis=0)
    m1 = ~(m0 | m2)
    xys_control1 = np.empty_like(xys_fix1)
    if np.any(m0):
        xys_control1[m0] = [xys_l45[np.argmin(d2b[...,i].max(1)), i] for i in np.nonzero(m0)[0]]
    if np.any(m1):
        xys_control1[m1] = [xys_l45[np.argmin(oob[:,i]), i] for i in np.nonzero(m1)[0]]
    if np.any(m2):
        rand_par = (rg.random(size=m2.sum()) < 0.5).astype(int)
        xys_control1[m2] = [xys_l45[p, i] for p, i in zip(rand_par, np.nonzero(m2)[0])]

    # - quality control
    for i, xys in enumerate((xys_fix1, xys_fix2, xys_control1, xys_control2)):
        if i < 2:
            # sanity check: control point must be equidistant to both fixations
            assert np.allclose(np.linalg.norm(xys_control1 - xys, axis=1), np.linalg.norm(vs, axis=1))

        d2b_ = np.array([
            np.max([xys[:,j]-im_size[j]/2, -im_size[j]/2-xys[:,j]], axis=0)
            for j in range(2)
        ])
        m = np.any(d2b_ >= 0, 0)
        d2b__ = d2b_[:,m].max(0)
        cond = ('Fixation 1', 'Fixation 2', 'Control 1 (equidist)', 'Control 2 (midpoint)')[i]
        print(f'{cond +":":<18}\t{m.mean()*100:4.1f}% out of image boundary', end='')
        if np.any(m):
            print(f'; d = {d2b__.mean():.1f} +/- {d2b__.std():.1f}')
        else:
            print()
        d2b__ = -(d2b_.max(0))
        print(f'\tmean min dist to bound: {d2b__.mean():.1f} +/- {d2b__.std():.1f} dva')

In [ ]:
ref_frames = np.array(('fix1', 'fix2', 'equidistant', 'midpoint')[:n_cond])
if select_saccades:
    xy_degs_conds = (xys_fix1, xys_fix2, xys_control1, xys_control2)
else:
    xy_degs_conds = (xys_fix2,)
    ref_frames = ref_frames[:1]

In [ ]:
group = analysis_name + '/fix_sel'
if select_saccades:
    save_results(
        group, np.array([fix1_sel, fix2_sel]),
        attrs=dict(dims=np.array(['fix_1_or_2', 'index'], dtype=np.bytes_)))
else:
    save_results(
        group, fix2_sel,
        attrs=dict(dims=np.array(['index'], dtype=np.bytes_)))

if select_saccades:
    group = analysis_name + '/control_points/'
    attrs = dict(dims=np.array(['index', 'xy'], dtype=np.bytes_), unit='dva', comment='Relative to image center')
    save_results(group+'equidistant', xys_control1, attrs=attrs)
    save_results(group+'midpoint', xys_control2, attrs=attrs)

# Get aligned responses

In [ ]:
with h5.File(sdf_path, 'r') as f:
    dset = f['sdf']
    all_unit_names = dset.attrs['unit_names'].astype(str)
    if unit_names is None:
        unit_names = all_unit_names
        unit_sel = slice(None)
        if 'unit_names' in f:
            copy_group(f, 'unit_names', analysis_name+'/unit_names')
    else:
        all_unit_names = list(all_unit_names)
        unit_sel = np.array([v in unit_names for v in all_unit_names])

    sdf = dset[()][:,unit_sel]

n_neur = sdf.shape[1]

In [ ]:
if t_win > 0:
    lat_df = pd.read_csv(latency_path).set_index('Session').loc[[sess_name]].set_index('Name')
    m = pd.Series(unit_names).isin(lat_df.index)
    assert m.all(), f'missing latency values for {(~m).sum()} of {m.size} units'
    assert not lat_df.index.has_duplicates
    lat_df = lat_df.loc[unit_names].reset_index()
    assert len(lat_df) == len(unit_names)
    lat_df['Index'] = np.arange(len(lat_df))
    lat_df['Latency'] = np.clip(lat_df['Latency'], 40, None)
    print('Num units using RF fit from each source:')
    print('\t' + '\n\t'.join(str(lat_df.groupby('Source').count()['Latency']).split('\n')[:-1]))
    lat_df.to_hdf(output_path, analysis_name+'/latency_per_unit', mode='a', format='table', complevel=9, complib='zlib')

In [ ]:
if t_aln == 'sacc_on':
    t0s = fix_df.iloc[fix1_sel][['Time', 'Duration']].values.sum(1)
else:
    t0s = fix_df.iloc[fix2_sel]['Time'].values

resps = np.empty_like(sdf, shape=(fix2_sel.size, ts.size, n_neur))

if t_win > 0:
    t_win_ = np.array([0, t_win])
    lat_groups = [(dt, df_['Index'].values) for dt, df_ in lat_df.groupby('Latency')]
    for i, t in enumerate(t0s):
        for dt, usel in lat_groups:
            s = slice(*np.round(t+dt+t_win_).astype(int))
            resps[i,0,usel] = sdf[s,usel].mean(0)
else:
    for i, t in enumerate(t0s):
        ts_ = np.round(t+ts).astype(int)
        resps[i] = sdf[ts_,:]

del sdf

In [ ]:
Y = standardize(resps)
Y.shape, Y.dtype

In [ ]:
save_results(analysis_name+'/unit_names', unit_names.astype(bytes))
save_results(analysis_name+'/mean_responses', resps.mean(0), attrs=dict(
    dims=np.array(['time', 'unit'], dtype=bytes),
    n_fix=resps.shape[0]))

# Define splits (group k-fold)

In [ ]:
imfns = fix_df.iloc[fix2_sel]['Image filename']

splits, train_mask = cv_split_by_image(
    imfns, n_splits,
    group_kfold=group_kfold, random_seed=random_seed)

In [ ]:
save_results(analysis_name+'/fix_is_train', train_mask, attrs=dict(
    dims=np.array(['split', 'fixation'], dtype=np.bytes_),
    random_seed=random_seed,
    group_kfold=group_kfold))

# Load pre-computed model reprs

In [ ]:
with h5.File(feats_path, 'r') as f:
    patch_locs_x = f['config/patch_grid/x_locs'][()]
    patch_locs_y = f['config/patch_grid/y_locs'][()]
    patch_step = float(f['config/patch_grid/step'][()])

    bg_feats = f['feats/bg'][()]

    all_md5s = f['md5'][()].astype(str)
    all_md5s = pd.Series(index=all_md5s, data=np.arange(len(all_md5s)), name='Index in file')
    idc = all_md5s.loc[md5_catalog['MD5'].values]
    md5_catalog['Index'] = np.arange(len(md5_catalog))

    patch_grid_feats = np.empty(shape=(idc.size,patch_locs_x.size,patch_locs_y.size)+bg_feats.shape, dtype=bg_feats.dtype)
    for ii, i in enumerate(idc.values):
        patch_grid_feats[ii] = f['feats/patch_grid'][i]  # shape (n_patches_x, n_patches_y,) + feats_shape

    copy_group(f, 'config', analysis_name+'/config/feats')

feats_shape = bg_feats.shape
print('Features shape:', feats_shape)
print('Patch-grid features shape:', patch_grid_feats.shape)

iims = np.array([
    md5_catalog.loc[(row['Image subdir'], row['Image filename']), 'Index']
    for i, (_, row) in enumerate(fix_df.iloc[fix2_sel].iterrows())])
patch_bins_x = np.concatenate([
    patch_locs_x-patch_step/2, [patch_locs_x[-1]+patch_step/2]])
patch_bins_y = np.concatenate([
    patch_locs_y-patch_step/2, [patch_locs_y[-1]+patch_step/2]])

In [ ]:
def recon_stim(*args, **kwargs):
    return get_patches_from_grid(
        *args, patch_bins_x=patch_bins_x, patch_bins_y=patch_bins_y, **kwargs)

# Main

In [ ]:
n_win = len(ts)
n_split = n_splits
n_loc = len(rf_locs)
n_unit = len(unit_names)

cv_corrs = np.full((n_cond, n_win, n_loc, n_loc, n_unit), np.nan, dtype=np.float32)
cv_r2s = np.full_like(cv_corrs, np.nan)
cv_corrs_per_split = np.full((n_cond, n_win, n_split, n_loc, n_loc, n_unit), np.nan, dtype=np.float32)
cv_r2s_per_split = np.full_like(cv_corrs_per_split, np.nan)

In [ ]:
for (ic, xy_degs), (iy, dy), (ix, dx) in tqdm(
        itertools.product(enumerate(xy_degs_conds), enumerate(rf_locs), enumerate(rf_locs)),
        total=n_cond*n_loc**2):

    X = standardize(recon_stim(iims, xy_degs+[dx,dy], patch_grid_feats, bg_feats))

    corr, r2, corr_pers, r2_pers = cv_ridge_predict_eval(X, Y, splits, ridge_alpha)

    cv_corrs[ic,:,iy,ix,:] = corr
    cv_r2s[ic,:,iy,ix,:] = r2
    cv_corrs_per_split[ic,:,:,iy,ix,:] = corr_pers
    cv_r2s_per_split[ic,:,:,iy,ix,:] = r2_pers

In [ ]:
dims = np.array(('ref_frame', 'time', 'split', 'rf_y', 'rf_x', 'unit'))
coords = dict((
    ('ref_frame', ref_frames),
    ('time', ts),
    ('rf_y', rf_locs),
    ('rf_x', rf_locs),
    ('unit', unit_names)))
attrs = dict(
    ifix_sel=ifix_sel,
    n_fix=fix2_sel.size,
    group_kfold=int(group_kfold),
    feat_shape=feats_shape,
    ridge_alpha=ridge_alpha,
    t_aln=t_aln)

q = lambda x: quantize(x, 3)
data_vars = {
    'corr': (dims[[0,1,3,4,5]], q(cv_corrs)),
    'r2': (dims[[0,1,3,4,5]], q(cv_r2s)),
    'corr_per_split': (dims, q(cv_corrs_per_split)),
    'r2_per_split': (dims, q(cv_r2s_per_split))}
dataset = xr.Dataset(data_vars, coords=coords, attrs=attrs)

compr = dict(zlib=True, complevel=9)
encoding = {
    k: dict(chunksizes=v.shape, **compr)
    for k, v in dataset.data_vars.items()}
dataset.to_netcdf(
    output_path, group=analysis_name+'/data',
    mode='a', engine='h5netcdf', encoding=encoding)

# Wrap up

In [ ]:
save_results(f'progress_report/{analysis_name}/all_done', True)

In [ ]:
%load_ext watermark
%watermark
%watermark -vm --iversions -rbg

# Basic visualization of results

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = {'Condition': [], 'X': [], 'Y': []}
for i, xys in enumerate(xy_degs_conds):
    if select_saccades:
        cond = ('Fixation 1', 'Fixation 2', 'Equidistant control', 'Midpoint control')[i]
    else:
        cond = 'Default'
    df['Condition'].append(np.full(len(xys), cond))
    df['X'].append(xys[:,0])
    df['Y'].append(xys[:,1])
for k in df.keys():
    df[k] = np.concatenate(df[k])
df = pd.DataFrame(df)
ax = sns.kdeplot(data=df, x='X', y='Y', hue='Condition', levels=3)
ax.set_title('Distribution of selected fixations')
ax.set_xlabel('X, dva')
ax.set_ylabel('Y, dva')
ax.set_aspect('equal')

In [ ]:
if select_saccades:
    sacc_vecs = (
        fix_df.iloc[fix2_sel][['Relative X', 'Relative Y']].values
        - fix_df.iloc[fix1_sel][['Relative X', 'Relative Y']].values)
    plt.hist2d(x=sacc_vecs[:,0], y=sacc_vecs[:,1], cmap='gist_gray_r')
    plt.xlabel('X, dva')
    plt.ylabel('Y, dva')
    plt.title('Distribution of selected saccade vectors')
    plt.gca().set_aspect('equal')

In [ ]:
plt.plot(ts, resps.mean((0,2)), '.-')
plt.xlabel(f'Time aligned to {t_aln}, ms')
plt.ylabel('Grand mean FR, spks/s');

In [ ]:
vr = abs(np.nanpercentile(np.nanmean(cv_corrs,-1), 99.5))

i = np.argmin(np.abs(ts - 200))
map_ = np.ma.masked_invalid(cv_corrs[0,i,:,:,:]).mean(-1).filled(np.nan)
cm = plt.imshow(map_, cmap='RdGy', origin='lower', extent=rf_extent, vmin=-vr, vmax=vr)
cb = plt.colorbar(cm)
cb.ax.set_ylabel('Model fit, Pearson\'s r')
plt.xlabel('X, dva')
plt.ylabel('Y, dva')
if select_saccades:
    plt.title(f'Fix 1-aligned model performance, {ts[i]} ms')
else:
    plt.title(f'Fixation-aligned model performance, {ts[i]} ms')

In [ ]:
if select_saccades:
    i = np.argmin(np.abs(ts - 200))
    map_ = np.ma.masked_invalid(cv_corrs[1,i,:,:,:]).mean(-1).filled(np.nan)
    cm = plt.imshow(map_, cmap='RdGy', origin='lower', extent=rf_extent, vmin=-vr, vmax=vr)
    cb = plt.colorbar(cm)
    cb.ax.set_ylabel('Model fit, Pearson\'s r')
    plt.xlabel('X, dva')
    plt.ylabel('Y, dva')
    plt.title(f'Fix 2-aligned model performance, {ts[i]} ms');

In [ ]:
if select_saccades:
    i = np.argmin(np.abs(ts))
    map_ = np.ma.masked_invalid(cv_corrs[1,i,:,:,:]).mean(-1).filled(np.nan)
    cm = plt.imshow(map_, cmap='RdGy', origin='lower', extent=rf_extent, vmin=-vr, vmax=vr)
    cb = plt.colorbar(cm)
    cb.ax.set_ylabel('Model fit, Pearson\'s r')
    plt.xlabel('X, dva')
    plt.ylabel('Y, dva')
    plt.title(f'Fix 2-aligned model performance, {ts[i]} ms');

In [ ]:
h = 3 * cv_corrs.shape[0]
w = min(h / cv_corrs.shape[0] * cv_corrs.shape[1], 20)
h = min(h, w / cv_corrs.shape[1] * cv_corrs.shape[0])
plt.figure(figsize=(w,h))
a = np.ma.masked_invalid(cv_corrs).mean(-1)
a = a.filled(np.nan)
a = a[:,:,::-1,:]  # flip y such that the first rows now correspond to larger y
a = np.hstack(np.hstack(a))
plt.imshow(a, cmap='RdGy', vmin=-vr, vmax=vr);
ax = plt.gca()
ax.set_xticks([])
ax.set_yticks([])
[v.set_visible(False) for v in ax.spines.values()]
plt.xlabel('Response window')
plt.ylabel('Reference frame')
plt.title('Mean 3D map per condition');